<a href="https://colab.research.google.com/github/sanujsriv/_transformer_models_/blob/main/Copy_of_EncoderDecoderModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- **Finish Cycle 2 by today (8/12)**

In [ ]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from transformers import DistilBertTokenizer, DistilBertForMaskedLM, AdamW
from transformers import AutoModel, AutoConfig, AutoTokenizer, BertTokenizer, BertModel
from transformers import DistilBertTokenizerFast, DistilBertModel, DistilBertConfig

from torch.utils.data import Dataset, DataLoader
import glob
import json


Using pretrained distilbert model
------------

*   Cycle 1
------------

1.   the reconstruction of the same reviews
2.   List item



*   Cycle 2
------------

1.   add a Neural network to get the combined review representation
2.   





In [ ]:
if not os.path.exists('/content/distilbert-base-uncased-finetuned-yelp-reviews/'):
#   !rm -rf /content/distilbert-base-uncased-finetuned-yelp-reviews.zip
    !wget -N -c -O distilbert-base-uncased-finetuned-yelp-reviews.zip https://www.dropbox.com/scl/fi/hv3f2x9wmew0znee0ur5o/distilbert-base-uncased-finetuned-yelp-reviews.zip?rlkey=dwm0ahwnpfanblnj492vw1zld&dl=0
    !unzip distilbert-base-uncased-finetuned-yelp-reviews.zip

In [ ]:
if not os.path.exists('/content/data_json.zip'):
  !wget -N -c -O data_json.zip https://www.dropbox.com/scl/fi/o2ddpudehrzjzjkdl7ela/data_json.zip?rlkey=okudk00osc5pfol1s66w5mtm8&st=m699q10z&dl=0
  !unzip data_json.zip -d /content/distilbert-base-uncased-finetuned-yelp-reviews/data

In [ ]:
os.chdir('/content/distilbert-base-uncased-finetuned-yelp-reviews/')
config = AutoConfig.from_pretrained('config.json')
tokenizer = AutoTokenizer.from_pretrained('.')

encoder_pretrained_model = AutoModel.from_pretrained('.', config=config)

encoder_pretrained_model.embeddings.word_embeddings.weight.requires_grad = False
# encoder_pretrained_model.embeddings.position_embeddings.weight.requires_grad = False
# encoder_pretrained_model.embeddings.token_type_embeddings.weight.requires_grad = False

decoder_pretrained_model = AutoModel.from_pretrained('.', config=config)

decoder_pretrained_model.embeddings.word_embeddings.weight.requires_grad = False
# decoder_pretrained_model.embeddings.position_embeddings.weight.requires_grad = False
# decoder_pretrained_model.embeddings.token_type_embeddings.weight.requires_grad = False

pretrained_embedding_layer = decoder_pretrained_model.embeddings.word_embeddings

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
decoder_masked_language_model = DistilBertForMaskedLM.from_pretrained('.', config=config)


Some weights of DistilBertForMaskedLM were not initialized from the model checkpoint at . and are newly initialized: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Yelp Data :-
n_docs = 16
# query_text = 'fried seafood'
def get_yelp_data(size):

    dir = '/content/distilbert-base-uncased-finetuned-yelp-reviews/data'

    os.chdir(dir)
    content = []

    for file in glob.glob("*.json"):
        with open(file,'r') as f:
            data = json.load(f)
            try:
                content.append(data[:n_docs])
            except:
                pass

    ### yelp
    dataset = "yelp"
    text_data = content
    documents = [doc['text'] for docs in text_data for doc in docs]
    if size=='full':
        size = len(documents)
    all_text = documents[:size] #+ [query_text]
    return all_text

# query_text = 'unemployment rate'
# all_text = flatten_list(documents) + [query_text]
documents = get_yelp_data(size=48)

In [ ]:
len(documents)

48

In [ ]:
# documents = ['Fantastic sliders!! Perfect size that you can try more than one!  Fried chicken slider, BLT slider, monte cristo and Cuban were all fantastic!  Now for Donuts to go!!',
#  'Walked here from our condo and found this awesome place! Delicious sliders...donuts...unique drinks... Awesome atmosphere and the staff was sooooooo friendly! Will definitely be back!!!!!',
#  'Great rich food,  but a lil bit pricey.  Everything tastes wonderful but nothing stays the same the menu is always changing.  The most unique donuts you will find.',
#  'I was in town for a short visit and decided to do some neighborhood exploration. I was in the garden district and wanted some cheap(er) lunch options. I found this place, had sliders and fries. I think both were very good (especially their chicken sliders). But there is more: the doughnuts. They are so so good: doughy perfectly pillowy and fresh. the flavor combinations are neat, creative. one last thing too: service is great. really helpful and nice staff.',
#  'LOVE THIS PLACE!! Visited on a flash trip to NOLA and loved the donuts. Cant wait to be backkk',
#  'Holy Donuts Batman!! One of my favorite spots for a quick sandwich and some delicious fries. Of course, being famous for their amazing donuts I have to sample a different one each time I visit. They are excellent! Also, have a coffee drink with your donut. Treat yo self!!',
#  'Yummy! I had the lemon ginger donut as dessert and had two sliders for my meal. I will say at lunch time on a Monday, getting those two sliders took over a half hour.  Only downside is these guys are waaaaay too busy.',
#  'I love this place! This past summer, I would go here almost every other day to buy a different kind of donut for my family to try. The atmosphere is great, service is great, and the food is amazing. Not only are their donuts tasty as heck, but their sliders are to die for as well. If you are a donut fan or just a sweet-tooth, you got to try this place. NOTE: The food truck that you might see around town, IMO, does not have the same quality. Go to the location on Magazine St.',
#  'We were heading out for brunch in the Garden District and literally made the cabby pull over as soon as we saw this place.  Hey - sounds like a bunch of good things rolled into one.  It was early, so we didn\'t have the sliders, but the donuts and coffee were exceptional.  The donuts looked overdone and loaded up with unnecessary ingredients at first, but I was wrong.  They were actually well balanced with good flavors.  They take coffee seriously too.  I tried their "Cold Nitro Brew" which seemed more like a Guinness than an iced coffee.  It wasn\'t exactly my style (so switched it for a latte instead) but I applaud them for trying to innovate.',
#  'I never write reviews on yelp and this place was so good that I felt the need to login and write a review. The poached pear donut was ridiculous. This place is doing everything right.']

In [ ]:
# Simulate 10 documents with the same input text for simplicity
# documents = ["This is a test sequence."] * 10
max_length = 150
n_docs_size = 16
encodings = tokenizer(documents, padding='max_length', max_length=max_length,
                          truncation=True, return_attention_mask=True,
                          return_tensors="pt")

input_ids , attention_mask = encodings['input_ids'] , encodings['attention_mask']
labels = input_ids.clone()  # Use the input IDs as labels for simplicity
added_vocab = tokenizer.get_added_vocab()
data_input_seqs = input_ids

data_input_seqs_item = data_input_seqs.reshape(data_input_seqs.shape[0] // n_docs_size, n_docs_size, data_input_seqs.shape[1])
attention_masks_item = attention_mask.reshape(attention_mask.shape[0] // n_docs_size, n_docs_size, attention_mask.shape[1])

In [ ]:
data_input_seqs_item.shape

torch.Size([3, 16, 150])

In [ ]:
data_input_seqs_item = data_input_seqs_item[:,:2,:]#.unsqueeze(1)
attention_masks_item = attention_masks_item[:,:2,:]#.unsqueeze(1)
documents_selected = np.asarray(documents).reshape(data_input_seqs.shape[0] // n_docs_size, n_docs_size)[:,:2]

1st cycle

In [ ]:
class TextDataset(Dataset):
    def __init__(self, input_seqs, target_seqs, attention_mask_seqs):
        self.input_seqs = input_seqs
        self.target_seqs = target_seqs
        self.attention_mask_seqs = attention_mask_seqs

    def __len__(self):
        return self.input_seqs.shape[0]

    def __getitem__(self, idx):
        return self.input_seqs[idx], self.target_seqs[idx], self.attention_mask_seqs[idx]

In [ ]:
SOS_id = added_vocab['[CLS]']
EOS_id = added_vocab['[SEP]']

In [ ]:
cos_sim = torch.nn.CosineSimilarity(dim=-1, eps=1e-08)

torch.cuda.empty_cache()

class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(input_dim, 1)

    def forward(self, x, dim):
        # x is expected to have shape (N, E), where N=10 and E=768
        attention_scores = self.attention_weights(x)  # Shape: (N, 1)
        attention_weights = torch.softmax(attention_scores, dim=dim)  # Shape: (N, 1)
        weighted_sum = torch.sum(attention_weights * x, dim=dim, keepdim=True)  # Shape: (1, E)
        return weighted_sum, attention_weights  # Shape: (1, E)

class Encoder(nn.Module):
    def __init__(self, encoder_distilbert, pretrained_embedding_layer):
        super(Encoder, self).__init__()
        self.encoder_distilbert = encoder_distilbert
        self.embedding_layer = pretrained_embedding_layer

    def forward(self, input_ids=None, inputs_embeds=None, attention_mask=None, labels=None):
        # input_ids_OH = F.one_hot(input_ids, num_classes=self.embedding_weight.shape[0]).float()
        if input_ids is not None:
            encoder_input_embeds = self.embedding_layer(input_ids)
            if(attention_mask is not None):
                outputs = self.encoder_distilbert(input_ids=input_ids,attention_mask=attention_mask)
            else:
                outputs = self.encoder_distilbert(input_ids=input_ids)
        else:
            encoder_input_embeds = inputs_embeds
            outputs = self.encoder_distilbert(inputs_embeds=encoder_input_embeds)
        hidden_states = outputs.last_hidden_state ## 2)
        return hidden_states


class Decoder(nn.Module):
    def __init__(self, decoder_distilbert):
        super(Decoder, self).__init__()

        self.hidden_dim_lmhead = int(decoder_distilbert.config.vocab_size / 2)
        self.decoder_distilbert = decoder_distilbert

        self.hidden_fc1 = nn.Linear(decoder_distilbert.config.dim, self.hidden_dim_lmhead)
        self.hidden_fc2 = nn.Linear(decoder_distilbert.config.dim, decoder_distilbert.config.vocab_size)
        self.lm_act = nn.ReLU()

        self.config = decoder_distilbert.config


    def forward(self, encoder_outputs, attention_mask=None, labels=None):
        outputs = self.decoder_distilbert(inputs_embeds=encoder_outputs, attention_mask=attention_mask) ## 1)
        hidden_states = outputs.last_hidden_state ## 2)
        # logits = self.lm_head(hidden_states) ##
        # logits_fc1 = self.hidden_fc1(hidden_states)
        # logits_fc1 = self.lm_act(logits_fc1)
        logits = self.hidden_fc2(hidden_states)

        return hidden_states, logits


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, attention, pretrained_embedding_layer, batch_size):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.batch_size = batch_size
        self.loss_fct = nn.CrossEntropyLoss(ignore_index=0)
        self.mean_fc = attention
        # self.mean_fc = nn.Linear(10,1)
        self.embedding_layer = pretrained_embedding_layer
        self.embed_size = pretrained_embedding_layer.weight.shape[-1]


    def forward(self, item_input_seq, item_attn_seq, labels=None):

        ### Loss Cycle 1 : Reconstruction Loss

        # Shift so that tokens < n predict n
        # ...: Keep all dimensions before the last two dimensions as they are.
        # :-1: Take all elements in the second-to-last dimension except the last one.
        # :: Keep all elements in the last dimension.

        item_batch_size = item_input_seq.shape[0] ### number of items (i)
        n_docs_size = item_attn_seq.shape[1] ### number of docs in items (n)

        ### [N, 16, 150] --> [N*16, 150]
        input_batch_ndocs = torch.flatten(item_input_seq, end_dim=1)
        attention_batch_ndocs = torch.flatten(item_attn_seq, end_dim=1)
        seq_len = input_batch_ndocs.shape[-1]
        output_encoder_hidden_states = self.encoder(input_ids=input_batch_ndocs,inputs_embeds=None, attention_mask=attention_batch_ndocs)
        hidden_states, logits = self.decoder(output_encoder_hidden_states,attention_mask=attention_batch)

        loss = None
        recon_loss = None
        if labels is not None:
            # shift_logits = logits[..., :-1, :].contiguous()
            # shift_labels = labels[..., 1:].contiguous()
            # recon_loss = self.loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            recon_loss = self.loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
######################################################################################################################

        ### Loss Cycle 2 : Review-Summarization Loss

        review_rep = output_encoder_hidden_states[:, 0, :]#[:, 0, :] #Assuming `[CLS]` token as the review representation  N x S x E > N x E


        # review_rep = review_rep*attention_batch_ndocs.unsqueeze(-1)
        review_rep = review_rep.view(item_batch_size, n_docs_size, 1 ,self.embed_size)


        comb_review_rep, attention_weights = self.mean_fc(review_rep,dim=1)
        comb_review_rep = comb_review_rep.squeeze(1)

        # comb_review_rep = review_rep.sum(1)#.unsqueeze(0)

        zero_stack = torch.zeros(item_batch_size,seq_len-1,self.embed_size).to(device)
        comb_review_rep = torch.cat((comb_review_rep,zero_stack),dim=1)
        attention_com_rev = torch.zeros((item_batch_size, seq_len), device=device)
        attention_com_rev[:,0] = 1
        summ_hidd_states, summ_logits = self.decoder(comb_review_rep,attention_mask = attention_com_rev) #SXE

        #summ_prob = F.gumbel_softmax(summ_logits, tau=0.1, eps=1e-20, hard=True) ### Gumbel #SxV

        #### we need a sequence for the summary
        # print(summ_logits.shape)
        review_summ_sim = None
        if labels is not None:
            summ_logits = summ_logits[:,:seq_len-1] ### 149 for inserting 101 later
            topk_probs, topk_indices = torch.topk(summ_logits, k=1, dim=-1)# CLS w1 w2....EOS w51 w52...
            topk_indices  = topk_indices.squeeze(-1)

            # print(topk_indices.shape)
            # print(torch.tensor([SOS_id], device=device).unsqueeze(0).repeat(2,1).shape)
            # xxx
            topk_indices = torch.cat((torch.tensor([SOS_id], device=device).unsqueeze(0).repeat(item_batch_size,1), topk_indices), dim=-1)
            EOS_idx = torch.where(topk_indices == EOS_id)

            summ_attn_mask = torch.ones((item_batch_size, seq_len), device=device)
            for i in (0,item_batch_size):
              if(i in EOS_idx[0]):
                summ_attn_mask[i,EOS_idx[-1][i]:] = 0
                topk_indices[i,EOS_idx[-1][i]+1:] = 0

            # summ_attn_mask = summ_attn_mask.unsqueeze(0)


            # topk_where_indices = torch.where(topk_indices == EOS_id).indices

            ##### [CLS]  w1  w2 ..... [SEP] <50> [PAD] [PAD] .... {150}
            #####   1    1   1  .....   1   <50>  0     0   .... {150}

            ## >>> create attention matrix <<<
            # print(summ_logits.shape,topk_indices.shape)
            summ_input_ids = topk_indices
            out_summ_enc_hidden = self.encoder(input_ids=summ_input_ids, attention_mask=summ_attn_mask)

            # Find the positions of all [MASK] tokens across the batch
            mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)

            # Forward pass through the model to get logits for all tokens in the batch
            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits

            # Gather logits for all mask positions (simultaneously)
            masked_token_logits = logits[mask_token_indices[0], mask_token_indices[1], :]

            # Get the predicted tokens for each [MASK] position by taking the argmax over the vocabulary
            predicted_token_ids = torch.argmax(masked_token_logits, dim=-1)

            # Replace the [MASK] tokens in input_ids with the predicted token ids
            input_ids[mask_token_indices[0], mask_token_indices[1]] = predicted_token_ids

            # Decode the sequences to get the final predicted texts for the batch
            predicted_sequences = [tokenizer.decode(input_id, skip_special_tokens=True) for input_id in input_ids]


            #summ_enc_input = torch.matmul(summ_logits, self.embedding_layer.weight)
            #out_summ_enc_hidden = self.encoder(input_ids=None,inputs_embeds=summ_enc_input)

            # # Get top-k predictions
            # topk_probs, topk_indices = torch.topk(summ_prob, k=50, dim=-1)

            # print(topk_indices)

            # #2.2 encoded summaries
            #out_summ_enc_hidden = out_summ_enc_hidden.squeeze(1)

            # #2.2 encoded summaries
            ## We compare the [CLS] token here     #### try the [SEP] token embedding here

            # print(out_summ_enc_hidden.shape, review_rep.shape)
            # print(review_rep[:,:,0,:].shape)
            # print((out_summ_enc_hidden[:,0,:].unsqueeze(1)).shape)
            # review_summ_sim = (1 - cos_sim(review_rep[:,:,0,:], out_summ_enc_hidden[:,0,:].unsqueeze(1))).mean(0).mean(0)

            # first_zero_index = (review_rep == 0).nonzero(as_tuple=True)#[0][0]
            # print(review_rep.shape,len(first_zero_index))
            # print(EOS_idx)
            # review_EOS_list = torch.where(input_ids == EOS_id)
            # out_summ_enc_EOS_list = torch.where(summ_input_ids == EOS_id)
            # print(attention_weights.shape)

            review_summ_sim = (1 - cos_sim(review_rep[:,:,0,:], out_summ_enc_hidden[:,0,:].unsqueeze(1))).mean(0).mean(0)

            # #############################
            # recon_loss = torch.tensor(0.0)
            # review_summ_sim = torch.tensor(0.0).to(device)

            loss =  recon_loss + review_summ_sim#.sum() +#+ similarity_loss + dissimilarity_loss
        return logits, summ_logits, loss, recon_loss, review_summ_sim #, recon_loss

In [ ]:
# Training loop
num_epochs = 1000
batch_size= 20 #### no. of items
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = encoder_pretrained_model.config.dim


# Initialize model
encoder = Encoder(encoder_pretrained_model, pretrained_embedding_layer)
decoder = Decoder(decoder_pretrained_model)
attention = Attention(input_dim)


# Optimizer
model = Seq2Seq(encoder, decoder, attention, pretrained_embedding_layer, batch_size)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# model = nn.DataParallel(model)
# torch.cuda.set_device(gpu)
# model.cuda(gpu)

model.to(device)
model.train()
dataset = TextDataset(data_input_seqs_item, data_input_seqs_item, attention_masks_item)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):
    batch_number = 1
    total_loss = 0
    total_recon_loss = 0
    total_review_summ_sim = 0
    for input_batch, label_batch, attention_batch in dataloader:
        # print(input_batch.shape)
        # print('batch number - ',batch_number)
        # input_batch = data_input_seq_item[i:i+batch_size]
        # attention_batch = attention_mask_item[i:i+batch_size]
        # label_batch = data_input_seq[i:i+batch_size]

        input_batch = input_batch.to(device)
        attention_batch = attention_batch.to(device)
        label_batch = label_batch.to(device)

        # print(input_batch.shape)
        optimizer.zero_grad()
        logits, summ_logits, loss, recon_loss, review_summ_sim = model(input_batch, attention_batch, label_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_recon_loss += recon_loss.item()
        total_review_summ_sim += review_summ_sim.item()
        batch_number = batch_number + 1

    avg_loss = total_loss / (len(documents) / batch_size)
    avg_recon_loss = total_recon_loss / (len(documents) / batch_size)
    avg_review_summ_sim = total_review_summ_sim / (len(documents) / batch_size)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}, recon_loss: {avg_recon_loss}, review_summ_sim: {avg_review_summ_sim}")

print("Training complete.")


Epoch 1/1000, Loss: 4.674805402755737, recon_loss: 4.318014780680339, review_summ_sim: 0.3567906220753988
Epoch 2/1000, Loss: 4.458985328674316, recon_loss: 4.24147645632426, review_summ_sim: 0.21750877300898236
Epoch 3/1000, Loss: 4.354768594106039, recon_loss: 4.182366927464804, review_summ_sim: 0.1724015548825264
Epoch 4/1000, Loss: 4.365275700887045, recon_loss: 4.114903211593628, review_summ_sim: 0.25037266314029694
Epoch 5/1000, Loss: 4.142593940099081, recon_loss: 4.068976640701294, review_summ_sim: 0.07361741736531258
Epoch 6/1000, Loss: 4.076898495356242, recon_loss: 4.02415672938029, review_summ_sim: 0.05274191498756409
Epoch 7/1000, Loss: 4.035144646962484, recon_loss: 3.995533386866252, review_summ_sim: 0.03961109245816867
Epoch 8/1000, Loss: 3.9925364653269453, recon_loss: 3.960100015004476, review_summ_sim: 0.03243646274010341
Epoch 9/1000, Loss: 3.962399959564209, recon_loss: 3.9341068267822266, review_summ_sim: 0.028293064484993618
Epoch 10/1000, Loss: 3.929492235183716

KeyboardInterrupt: 

In [ ]:
model.eval()
generated_texts = []
generated_texts_summ = []
# reverse_id_mapping = {v: k for k, v in id_mapping.items()}

with torch.no_grad():
    for input_batch, label_batch, attention_batch in dataloader:
        input_batch = input_batch.to(device)
        attention_batch = attention_batch.to(device)

        logits, summ_logits, _, _, _ = model(input_batch, attention_batch)

        # Apply softmax to get probabilities
        softmax_logits = F.softmax(logits, dim=-1)

        # Get top-k predictions
        topk_probs, topk_indices = torch.topk(softmax_logits, k=5, dim=-1)

        for j in range(topk_indices.size(0)):
            generated_tokens = []
            for step in range(topk_indices.size(1)):
                token_index = topk_indices[j, step, 0].item()  # Choose top 1
                if token_index == EOS_id:
                  break
                else:
                  generated_tokens.append(token_index) #reverse_id_mapping[token_index]

            decoded_text = tokenizer.decode(generated_tokens, skip_special_tokens=False)
            generated_texts.append(decoded_text)

        softmax_summ_logits = F.softmax(summ_logits, dim=-1)

        # Get top-k predictions
        topk_probs_summ, topk_indices_summ = torch.topk(softmax_summ_logits, k=5, dim=-1)

        for j in range(topk_indices_summ.size(0)):
            generated_tokens_summ = []
            for step in range(topk_indices_summ.size(1)):
                token_index = topk_indices_summ[j, step, 0].item()  # Choose top 1
                if token_index == EOS_id:
                    break
                else:
                    generated_tokens_summ.append(token_index) #reverse_id_mapping[token_index]

            decoded_text_summ = tokenizer.decode(generated_tokens_summ, skip_special_tokens=False)
            generated_texts_summ.append(decoded_text_summ)

review_count = 2
out_doc_size = 2
review_item = 1
original_docs = documents_selected.flatten()
# for original, generated, summary in zip([documents[0],documents[16],documents[32]], generated_texts,generated_texts_summ):

for original, generated in zip(original_docs, generated_texts):
    if(review_count==out_doc_size):
        print("*"*10)
        print("Item :",review_item)
        print("*"*10)
        print()
        print('#'*150)
        print("Summary:", generated_texts_summ[review_item-1])
        print('#'*150)
        print()
        review_item = review_item + 1
        review_count = 0
    review_count = review_count + 1
    print(review_count,") Original Text:", original)
    print(review_count,") Generated Text:", generated)
    print()
#

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch

# Load pretrained DistilBERT model for masked language modeling
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

# Set the model in evaluation mode
model.eval()


In [ ]:
input_text = "The [MASK] [MASK] [MASK] [MASK] [MASK]."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Get the position of the masked tokens
mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)[1]

def predict_masked_tokens(input_ids, model, tokenizer, num_masks):
    for _ in range(num_masks):  # Iterate over the number of masked tokens
        mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

        print(len(mask_token_index),mask_token_index)
        # Get model predictions for masked tokens
        with torch.no_grad():
            outputs = model(input_ids)

        logits = outputs.logits
        print(logits.shape)

        mask_token_logits = logits[0, mask_token_index[0], :]
        print(mask_token_logits.shape)

        # Select the token with the highest probability
        predicted_token_id = torch.argmax(mask_token_logits, dim=-1)

        # Update the input_ids with the predicted token
        print(input_ids.shape)
        input_ids[0, mask_token_index[0]] = predicted_token_id
        print(input_ids)

    return input_ids

# Predict masked tokens and update the input_ids
num_masks = (input_ids == tokenizer.mask_token_id).sum().item()
predicted_input_ids = predict_masked_tokens(input_ids, model, tokenizer, num_masks)

# Decode the final sequence
predicted_sequence = tokenizer.decode(predicted_input_ids[0], skip_special_tokens=True)
print("Predicted sequence: ", predicted_sequence)


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForMaskedLM

# Load pretrained DistilBERT model for masked language modeling
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

# Set the model in evaluation mode
model.eval()

# Example input with multiple masked tokens
input_text = "The [MASK] [MASK] [MASK] [MASK] [MASK]."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Find the positions of all [MASK] tokens
mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)

# Predict all masked tokens at once
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# Gather logits for all mask positions
masked_token_logits = logits[0, mask_token_indices[1], :]

# Get the predicted tokens for each [MASK] position by taking the argmax over the vocabulary
predicted_token_ids = torch.argmax(masked_token_logits, dim=-1)

# Replace the [MASK] tokens in input_ids with the predicted token ids
input_ids[0, mask_token_indices[1]] = predicted_token_ids

# Decode the sequence to get the final predicted text
predicted_sequence = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print(f"Predicted sequence: {predicted_sequence}")


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForMaskedLM

# Load pretrained DistilBERT model for masked language modeling
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

# Set the model to evaluation mode
model.eval()

# Example batch of input texts with multiple masked tokens
input_texts = [
    "The [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK].",
    "Machine [MASK] [MASK]  [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .",
    "The [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]."
]

# Tokenize the batch of input texts
encoded_inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)

# Extract the input_ids and attention mask
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Display the tokenized versions of the inputs (before mask prediction)
for idx, input_id in enumerate(input_ids):
    print(f"Tokenized input {idx+1} before prediction: ", tokenizer.convert_ids_to_tokens(input_id))

# Find the positions of all [MASK] tokens across the batch
mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)

# Forward pass through the model to get logits for all tokens in the batch
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Gather logits for all mask positions (simultaneously)
masked_token_logits = logits[mask_token_indices[0], mask_token_indices[1], :]

# Get the predicted tokens for each [MASK] position by taking the argmax over the vocabulary
predicted_token_ids = torch.argmax(masked_token_logits, dim=-1)

# Replace the [MASK] tokens in input_ids with the predicted token ids
input_ids[mask_token_indices[0], mask_token_indices[1]] = predicted_token_ids

# Decode the sequences to get the final predicted texts for the batch
predicted_sequences = [tokenizer.decode(input_id, skip_special_tokens=True) for input_id in input_ids]

# Print each predicted sequence in the batch
for idx, sequence in enumerate(predicted_sequences):
    print(f"Predicted sequence {idx+1}: {sequence}")


Tokenized input 1 before prediction:  ['[CLS]', 'the', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '.', '[SEP]', '[PAD]', '[PAD]']
Tokenized input 2 before prediction:  ['[CLS]', 'machine', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '.', '[SEP]', '[PAD]', '[PAD]']
Tokenized input 3 before prediction:  ['[CLS]', 'the', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '.', '[SEP]']
Predicted sequence 1: the codex is contains with with withδ ).
Predicted sequence 2: machine learning : with with with withו }.
Predicted sequence 3: the codexυεεεεελλ ).


In [ ]:
# model.eval()
# generated_texts = []

# with torch.no_grad():
#     for i in range(0, len(documents), batch_size):
#         input_batch = input_ids[i:i+batch_size]
#         input_batch = input_batch.to(device)
#         logits, _ = model(input_batch)

#         # Apply softmax to get probabilities
#         softmax_logits = F.softmax(logits, dim=-1)

#         # Get top-k predictions
#         topk_probs, topk_indices = torch.topk(softmax_logits, k=5, dim=-1)

#         for j in range(input_batch.size(0)):
#             generated_tokens = []
#             for step in range(input_batch.size(1)):
#                 token_index = topk_indices[j, step, 0].item()  # Choose top 1
#                 generated_tokens.append(token_index)

#             decoded_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
#             generated_texts.append(decoded_text)

# for original, generated in zip(documents, generated_texts):
#     print("Original Text:", original)
#     print("Generated Text:", generated)
#     print()


In [ ]:
id_vocab[1010]

In [ ]:
topk_probs , topk_indices

In [ ]:
id_vocab = {v:k for k,v in vocab.items()}
id_vocab[1]

In [ ]:
token_index

2nd cycle

In [ ]:
# - use this model_with_lm_head as decoder for summary decoder
# - add a NN for

## EncoderDecoderModel

In [ ]:
# from transformers import EncoderDecoderModel, DistilBertTokenizer, DistilBertModel, BertModel

# # Load tokenizer
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# # Load encoder (DistilBERT)
# encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')

# # Load decoder (BERT)
# decoder = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
# class DistilBERTWithLMHead(nn.Module):
#     def __init__(self, distilbert):
#         super(DistilBERTWithLMHead, self).__init__()
#         self.distilbert = distilbert
#         self.lm_head = nn.Linear(distilbert.config.dim, distilbert.config.vocab_size)

#     def forward(self, input_ids, encoder_hidden_states=None, encoder_attention_mask=None, labels=None):
#         outputs = self.distilbert(input_ids=input_ids, encoder_hidden_states=encoder_hidden_states, encoder_attention_mask=encoder_attention_mask)
#         hidden_states = outputs[0]
#         logits = self.lm_head(hidden_states)

#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss()
#             shift_logits = logits[..., :-1, :].contiguous()
#             shift_labels = labels[..., 1:].contiguous()
#             loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

#         return logits, loss

# # Load the DistilBERT model
# distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
# decoder = DistilBERTWithLMHead(distilbert)


In [ ]:
from transformers import EncoderDecoderModel, BertForMaskedLM

# Create the encoder-decoder model
# decoder = BertForMaskedLM.from_pretrained('bert-base-uncased
# decoder = BertModel.from_pretrained('bert-base-uncased')
# Create the encoder-decoder model
model = EncoderDecoderModel(encoder=pretrained_model, decoder=model_with_lm_head)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size



In [ ]:
# Simulate 10 documents with the same input text for simplicity
documents = ["This is a test sequence."] * 10
inputs = tokenizer(documents, return_tensors='pt', padding=True, truncation=True)
labels = tokenizer(documents, return_tensors='pt', padding=True, truncation=True).input_ids


In [ ]:
import torch
import torch.optim as optim

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 10
batch_size = 10

model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for i in range(0, len(documents), batch_size):
        input_batch = inputs.input_ids[i:i+batch_size]
        label_batch = labels[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(input_ids=input_batch, labels=label_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / (len(documents) / batch_size)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

print("Training complete.")


In [ ]:
model.eval()
generated_texts = []

with torch.no_grad():
    for i in range(0, len(documents), batch_size):
        input_batch = inputs.input_ids[i:i+batch_size]
        generated_ids = model.generate(input_batch, max_length=50)

        for token_ids in generated_ids:
            decoded_text = tokenizer.decode(token_ids, skip_special_tokens=True)
            generated_texts.append(decoded_text)

for original, generated in zip(documents, generated_texts):
    print("Original Text:", original)
    print("Generated Text:", generated)
    print()
